In [1]:
import json
from copy import deepcopy
from datetime import datetime, timedelta

import pandas as pd

from Core.Dexter.Infrastructure.Domain.Breakdowns import BreakdownMetadataBase, BreakdownBase
from Core.Dexter.Infrastructure.Domain.LevelEnums import LevelEnum
from Core.Tools.Misc.Constants import DEFAULT_DATETIME
from Core.Tools.Misc.ObjectManipulators import extract_class_attributes_values
from FacebookDexter.BackgroundTasks.Startup import Startup
from FacebookDexter.Engine.Algorithms.AlgorithmsEnum import FacebookAlgorithmsEnum
from FacebookDexter.Engine.Algorithms.FuzzyRuleBasedSingleMetricOptimization.FacebookRuleBasedSingleMetricFuzzyfierFactory import \
    FacebookRuleBasedSingleMetricFuzzyfierFactory
from FacebookDexter.Engine.Algorithms.FuzzyRuleBasedSingleMetricOptimization.Metrics.FacebookAvailableSingleMetricEnum import \
    FacebookAvailableSingleMetricEnum
from FacebookDexter.Infrastructure.Domain.Breakdowns import FacebookBreakdownEnum, FacebookActionBreakdownEnum
from FacebookDexter.Infrastructure.Domain.Metrics.FacebookMetricCalculator import FacebookMetricCalculator
from FacebookDexter.Infrastructure.PersistanceLayer.FacebookDexterMongoRepository import FacebookDexterMongoRepository
from FacebookDexter.Infrastructure.PersistanceLayer.RecommendationsRepository import RecommendationsRepository

In [2]:
def get_last_30days_raw_data(calculator=None, dates=None):
    raw_values = []
    for current_date in dates:
        raw_value = calculator.raw_value(date_start=current_date)
        entry = {
            'datetime': current_date,
            'value': raw_value
        }
        raw_values.append(deepcopy(entry))
    return raw_values

def get_raw_value(calculator=None, current_date=None):
    current_date = current_date.strftime(DEFAULT_DATETIME)
    raw_value = calculator.raw_value(date_start=current_date)
    return raw_value

def generate_last_30days(date_stop=None):
    dates = []
    for time_interval in range(0, 30):
        current_date = (date_stop - timedelta(days=time_interval)).strftime(DEFAULT_DATETIME)
        dates.append(current_date)
    return dates

def compute_trend_30days(calculator=None, date_stop=None):
    date_start = (date_stop - timedelta(days=30)).strftime(DEFAULT_DATETIME)
    date_stop = date_stop.strftime(DEFAULT_DATETIME)
    trend, _ = calculator.fuzzy_trend(date_start=date_start, date_stop=date_stop)
    return trend

def compute_variance(calculator=None, time_interval=None, date_stop=None):
    date_start = (date_stop - timedelta(days=time_interval['value'])).strftime(DEFAULT_DATETIME)
    date_stop = date_stop.strftime(DEFAULT_DATETIME)
    variance = calculator.variance(date_start=date_start, date_stop=date_stop)
    return variance

def compute_average(calculator=None, time_interval=None, date_stop=None):
    date_start = (date_stop - timedelta(days=time_interval['value'])).strftime(DEFAULT_DATETIME)
    date_stop = date_stop.strftime(DEFAULT_DATETIME)
    average = calculator.average(date_start=date_start, date_stop=date_stop)
    return average

def compute_percentage_difference(calculator=None, time_interval=None, date_stop=None):
    date_start = (date_stop - timedelta(days=time_interval['value'])).strftime(DEFAULT_DATETIME)
    date_stop = date_stop.strftime(DEFAULT_DATETIME)
    percentage_difference = calculator.percentage_difference(date_start=date_start, date_stop=date_stop)
    return percentage_difference

def prepare_single_metric_export_data(recommendation=None, calculator=None):

    latest_datetime = datetime.strptime(recommendation['created_at'], '%Y-%m-%dT%H:%M:%S')
    interval_30days = generate_last_30days(date_stop=latest_datetime)
    last_30days_data = get_last_30days_raw_data(calculator, dates=interval_30days)
    data = {
        'account_id': recommendation['ad_account_id'].split("_")[1],
        'structure_name': recommendation['structure_name'],
        'structure_id': recommendation['structure_id'],
        'time_interval': recommendation['time_interval']['value'],
        'recommendation': recommendation['template'],
        '30 days time interval': [e['datetime'] for e in last_30days_data],
        recommendation['metrics'][0]['display_name']: [e['value'] for e in last_30days_data],
        'trend': compute_trend_30days(calculator, date_stop=latest_datetime),
        'variance': compute_variance(calculator, recommendation['time_interval'], date_stop=latest_datetime),
        'created_at': recommendation['created_at'],
        'last_data_point_date': latest_datetime,
        'raw_value_when_created': get_raw_value(calculator, latest_datetime),
        'percentage_difference': compute_percentage_difference(calculator, time_interval=recommendation['time_interval'], date_stop=latest_datetime),
        'mean': compute_average(calculator, time_interval=recommendation['time_interval'], date_stop=latest_datetime),
        'confidence_level': recommendation['confidence'],
        'status': recommendation['status']
    }
    return data

def prepare_single_metric_csv_raw_data(recommendations=None, repository=None):
    data = []
    for recommendation in recommendations:
        # get the level
        level = LevelEnum.get_enum_by_value(recommendation['level'].lower())

        # initialize the metric
        metric = next(filter(lambda x: x.name == recommendation['metrics'][0]['name'],
                             extract_class_attributes_values(FacebookAvailableSingleMetricEnum)), None)
        # recreate breakdown metadata
        breakdown_metadata = BreakdownMetadataBase(breakdown=FacebookBreakdownEnum(recommendation['breakdown']['name'].upper()),
                                                   action_breakdown=FacebookActionBreakdownEnum(recommendation['action_breakdown']['name'].upper()))

        # get the fuzzyfier factory for the current algorithm and level
        fuzzyfier_factory = FacebookRuleBasedSingleMetricFuzzyfierFactory.get(algorithm_type=FacebookAlgorithmsEnum.DEXTER_FUZZY_INFERENCE,
                                                                              level=level)

        # create a generic calculator
        calculator = (FacebookMetricCalculator().
                      set_metric(metric).
                      set_facebook_id(recommendation['structure_id']).
                      set_level(level).
                      set_fuzzyfier_factory(fuzzyfier_factory).
                      set_repository(repository).
                      set_time_interval(recommendation['time_interval']['value']).
                      set_breakdown_metadata(breakdown_metadata).
                      set_debug_mode(False))

        # prepare the raw data for the current recommendation
        raw_data = prepare_single_metric_export_data(recommendation, calculator)
        data.append(deepcopy(raw_data))
    return data

In [3]:
# setup parameters for generating the spreadsheet
start_date = '2020-08-14'
end_date = datetime.now().strftime(DEFAULT_DATETIME)

output_file = 'facebook-dexter-recommendations-20200817.csv'

# ====== CHANGE THIS DEPENDING ON YOUR LOCAL SETUP ======
# path to the FacebookDexter config file.
config_file_path = '/Users/luchicla/Work/Filed/Filed.Python/FacebookDexter/BackgroundTasks/Config/Settings/app.settings.dev.json'

with open(config_file_path, 'r') as config_file:
    app_config = json.load(config_file)

startup = Startup(app_config)

recommendations_repository = RecommendationsRepository(config=startup.mongo_config,
                                                       database_name=startup.mongo_config.recommendations_database_name,
                                                       collection_name=startup.mongo_config.recommendations_collection_name)
data_repository = FacebookDexterMongoRepository(config=startup.mongo_config,
                                                database_name=startup.mongo_config.insights_database)

In [4]:
# get recommendations between start_date and end_date
recommendations = recommendations_repository.get_last_updated(key_name='created_at', key_value=start_date)
#csv_data = prepare_single_metric_csv_raw_data(recommendations, data_repository)
#pd.to_csv(csv_data, output_file)

In [5]:
data = prepare_single_metric_csv_raw_data([recommendations[0]])



ValueError: 'NONE' is not a valid FacebookBreakdownEnum